In [3]:
import torch
import torch.nn.functional as F
import json
import cv2
import numpy as np

In [44]:
def flow_warp(x, flow, interp_mode='bilinear', padding_mode='zeros'):
    """Warp an image or feature map with optical flow
    Args:
        x (Tensor): size (N, C, H, W)
        flow (Tensor): size (N, H, W, 2), normal value
        interp_mode (str): 'nearest' or 'bilinear'
        padding_mode (str): 'zeros' or 'border' or 'reflection'
    Returns:
        Tensor: warped image or feature map
    """
    assert x.size()[-2:] == flow.size()[1:3]
    B, C, H, W = x.size()
    # mesh grid
    grid_y, grid_x = torch.meshgrid(torch.arange(0, H), torch.arange(0, W))
    grid_y, grid_x = grid_y.type_as(x), grid_x.type_as(x)
    grid = torch.stack((grid_x, grid_y), 2).float()  # W(x), H(y), 2
    grid.requires_grad = False
    vgrid = grid + flow
    # scale grid to [-1,1]
    vgrid_x = 2.0 * vgrid[:, :, :, 0] / max(W - 1, 1) - 1.0
    vgrid_y = 2.0 * vgrid[:, :, :, 1] / max(H - 1, 1) - 1.0
    vgrid_scaled = torch.stack((vgrid_x, vgrid_y), dim=3)
    output = F.grid_sample(x, vgrid_scaled, mode=interp_mode, padding_mode=padding_mode)
    return output

In [27]:
with open('./113.json') as json_file:
    json_f = json.load(json_file)
img = cv2.imread('./frame112.png')

In [28]:
img.shape

(720, 1280, 3)

In [29]:
new_img = np.rollaxis(img, 2, 0)  

In [30]:
new_img.shape

(3, 720, 1280)

In [31]:
new_img = new_img[np.newaxis,:]

In [32]:
new_img.shape

(1, 3, 720, 1280)

In [33]:
arr = np.zeros((img.shape[0],img.shape[1],2))

for dic in json_f:
    width = int(dic["width"] / 2)
    height = int(dic["height"] / 2)
    x = dic['src_x']
    y = dic['src_y']
    arr[y-height:y+height,x-width:x+width,0] = dic["dx"]
    arr[y-height:y+height,x-width:x+width,1] = dic["dy"]

In [34]:
arr = arr[np.newaxis,:]
print(arr.shape)

(1, 720, 1280, 2)


In [51]:
new_img_tensor = torch.from_numpy(new_img).type(torch.DoubleTensor)
arr_tensor = torch.from_numpy(arr).type(torch.DoubleTensor)

In [52]:
output = flow_warp(new_img_tensor,arr_tensor)

In [53]:
output.shape

torch.Size([1, 3, 720, 1280])

In [64]:
output_np = output.numpy()[0]
output_np = np.moveaxis(output_np, 0, 2)  

In [65]:
output_np.shape

(720, 1280, 3)

In [66]:
cv2.imwrite('image_flow.png',output_np) 

True